In [60]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
import time
from selenium.webdriver import ActionChains
import sys
import time
import os

In [61]:
dfUploadParameters = pd.read_excel(r'D:\Users\vbhati01\FileConsolidation\Prerequisites\Required Process Parameters.xlsx',header = 5)

In [62]:
dfUploadParameters

,Type,Field,Source,Value,Comments- We will remove these and replace with more generic comments for the user later
0,Generic,Username,TamroFI,samuel.gauvain,NaN
1,NaN,Password,TamroFI,Calypso2020,NaN
2,NaN,Username,Pharmaview,sgauvain@amgen.com,NaN
3,NaN,Password,Pharmaview,Covid19reallysucks,NaN
4,NaN,Path,TamroFI,D:\Users\vbhati01\FileConsolidation\Daily\Tamr...,NaN
5,NaN,Path,Pharmaview Daily Primary Sector,D:\Users\vbhati01\FileConsolidation\Daily\Phar...,NaN
6,NaN,Path,Pharmaview Daily Hospital Sector,D:\Users\vbhati01\FileConsolidation\Daily\Phar...,NaN
7,NaN,Path,Pharmaview Weekly Primary Sector,D:\Users\vbhati01\FileConsolidation\Daily\Phar...,NaN
8,NaN,Path,Pharmaview Weekly Hospital Sector,D:\Users\vbhati01\FileConsolidation\Daily\Phar...,NaN
9,NaN,Path,MoviantoDK,D:\Users\vbhati01\FileConsolidation\Daily\Movi...,NaN


In [63]:
username = dfUploadParameters[(dfUploadParameters['Field'].str.lower().str.strip()=='username') & (dfUploadParameters['Source'].str.lower().str.strip()=='tamrofi')]['Value'].item()

In [64]:
password = dfUploadParameters[(dfUploadParameters['Field'].str.lower().str.strip()=='password') & (dfUploadParameters['Source'].str.lower().str.strip()=='tamrofi')]['Value'].item()

In [65]:
downloadDirectory = dfUploadParameters[(dfUploadParameters['Field'].str.lower().str.strip()=='path') & (dfUploadParameters['Source'].str.lower().str.strip()=='tamrofi')]['Value'].item()

In [66]:
downloadDirectory

'D:\\Users\\vbhati01\\FileConsolidation\\Daily\\TamroFi\\\\'

In [67]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": downloadDirectory,  
             "directory_upgrade": True}
options.add_experimental_option('prefs',prefs)

In [68]:
chromeDriver = dfUploadParameters[(dfUploadParameters['Source'].str.lower().str.strip()=='chromedriver')]['Value'].item()

In [69]:
chromeDriver

'D:\\Users\\vbhati01\\FileConsolidation\\Prerequisites\\chromedriver.exe'

CSS SELECTOR FOR DAILY SALES FIA : #QvAjaxToolbar-left > li.cf.ctx-menu-action-Bookmarks > select

In [70]:
driver = webdriver.Chrome(chromeDriver,options = options)
# driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
# params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': "C:/Users/venug/Downloads"}}
# command_result = driver.execute("send_command", params)
initialCount = len(os.listdir(downloadDirectory))
newCount = initialCount
retries = 0

while(initialCount == newCount and retries<5):
    try:
        driver.get('https://bright.tamro.fi/qlikview/FormLogin.htm')
        driver.find_element_by_id("username").send_keys(username)
        driver.find_element_by_id("password").send_keys(password)
        driver.find_element(By.XPATH,'//*[@id="loginForm"]/input').click()
        timeout = 500
        locator = '//*[@id="appList2"]/span[1]/span[2]/a[2]'
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
        driver.find_element(By.XPATH,'//*[@id="appList2"]/span[1]/span[2]/a[2]').click()
        time.sleep(10)
        driver.switch_to.window(driver.window_handles[-1])
        timeout = 500
        # locator = '//*[@id="QvAjaxToolbar-left"]/li[19]/select'
        # WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
        # driver.find_element(By.XPATH,'//*[@id="QvAjaxToolbar-left"]/li[19]/select').send_keys('Daily Sales Fia')
        locator = '#QvAjaxToolbar-left > li.cf.ctx-menu-action-Bookmarks > select'
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CSS_SELECTOR, locator)))
        time.sleep(10)
        driver.find_element(By.CSS_SELECTOR,'#QvAjaxToolbar-left > li.cf.ctx-menu-action-Bookmarks > select').send_keys('Daily Sales Fia')
        timeout = 500
        locator = '//*[@id="165"]/div[2]/div[1]/div[1]/div[1]/div/div[1]'
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
        dateTable = driver.find_element(By.XPATH,'//*[@id="165"]/div[2]/div[1]/div[1]/div[1]/div/div[1]')
        actionChains = ActionChains(driver)
        time.sleep(5)
        actionChains.context_click(dateTable).perform()
        timeout = 500
        locator = '/html/body/ul/li[4]/a'
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
        driver.find_element(By.XPATH,'/html/body/ul/li[4]/a').click()
        # driver.switch_to.window(driver.window_handles[-1])
        time.sleep(20)
    
    except Exception as e:
        print(e)
        print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
        driver.close()
    retries+=1
    time.sleep(30)
    newCount = len(os.listdir(downloadDirectory))

if(retries<5):
    print("File downloaded")
    logFilePath = dfUploadParameters[(dfUploadParameters['Field'].str.lower().str.strip()=='path') & (dfUploadParameters['Source'].str.lower().str.strip()=='logfile')]['Value'].item()
    dfLogFile = pd.read_excel(logFilePath,index_col = "SNo.")
    dfLogFile.loc[dfLogFile['Script']=='TamroDailyData','Executed']= 1
    dfLogFile.to_excel(logFilePath)
else:
    raise Exception("Download Error")
    #print("Download error")
    
driver.quit()

File downloaded


FileCreateError: [Errno 13] Permission denied: 'D:\\Users\\vbhati01\\FileConsolidation\\Daily\\logFile.xlsx'